In [19]:
# Imports
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations

# Read in the data
df_2023 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2023-05.parquet")
df_2022 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2022-05.parquet")
df_2021 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2021-05.parquet")
df_2020 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2020-05.parquet")
df_2019 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2019-05.parquet")
df_2018 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2018-05.parquet")
df_2017 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2017-05.parquet")
df_2016 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2016-05.parquet")
df_2015 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2015-05.parquet")
df_2014 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2014-05.parquet")
df_2013 = pd.read_parquet("/Users/jesperberglund/Desktop/TaxiDataMay/yellow_tripdata_2013-05.parquet")

# Rename columns to include year for each dataframe and concatenate them into one dataframe with all years
dataframes = [df_2023, df_2022, df_2021, df_2020, df_2019, df_2018, df_2017, df_2016, df_2015, df_2014, df_2013]
years = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013]
for df, year in zip(dataframes, years):
    df.columns = [f'{col}{year}' for col in df.columns]
df_total = pd.concat(dataframes, axis=1)

print(df_total.columns)

Index(['VendorID2023', 'tpep_pickup_datetime2023', 'tpep_dropoff_datetime2023',
       'passenger_count2023', 'trip_distance2023', 'RatecodeID2023',
       'store_and_fwd_flag2023', 'PULocationID2023', 'DOLocationID2023',
       'payment_type2023',
       ...
       'payment_type2013', 'fare_amount2013', 'extra2013', 'mta_tax2013',
       'tip_amount2013', 'tolls_amount2013', 'improvement_surcharge2013',
       'total_amount2013', 'congestion_surcharge2013', 'airport_fee2013'],
      dtype='object', length=209)


In [ ]:
# Export a random sample of 100,000 rows to a csv file
columns_to_export = [f'{col}{year}' for year in years for col in ['tip_amount', 'tpep_pickup_datetime', 'PULocationID', 'DOLocationID']]
# The length of the "shortest" column is 2507109:
print(len(df_2021['tip_amount2021']))
random_indices = np.sort(np.random.choice(2507109, size=100_000, replace=False))
df_total[columns_to_export].iloc[random_indices].to_csv('df_tip.csv')

In [24]:
# 5. maj 2013 og 2017
df_5th_may_2013 = df_2013[['tip_amount2013', 'PULocationID2013', 'DOLocationID2013', 'tpep_pickup_datetime2013']]
df_5th_may_2017 = df_2017[['tip_amount2017','PULocationID2017', 'DOLocationID2017', 'tpep_pickup_datetime2017']]

# Convert 'tpep_pickup_datetime' columns to datetime
df_5th_may_2013['tpep_pickup_datetime2013'] = pd.to_datetime(df_5th_may_2013['tpep_pickup_datetime2013'])
df_5th_may_2017['tpep_pickup_datetime2017'] = pd.to_datetime(df_5th_may_2017['tpep_pickup_datetime2017'])

# Filter rows where day is 5 for both 2013 and 2017
df_5th_may_2013 = df_5th_may_2013[((df_5th_may_2013['tpep_pickup_datetime2013'].dt.day == 5))]
df_5th_may_2017 = df_5th_may_2017[((df_5th_may_2017['tpep_pickup_datetime2017'].dt.day == 5))]

# Export to csv
df_5th_may_2013.to_csv('df_5th_may_2013.csv')
df_5th_may_2017.to_csv('df_5th_may_2017.csv')



/var/folders/0c/bp6_ntbd5rj6j3w7h1dg5xvm0000gn/T/ipykernel_42983/2819314562.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5th_may_2013['tpep_pickup_datetime2013'] = pd.to_datetime(df_5th_may_2013['tpep_pickup_datetime2013'])
/var/folders/0c/bp6_ntbd5rj6j3w7h1dg5xvm0000gn/T/ipykernel_42983/2819314562.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5th_may_2017['tpep_pickup_datetime2017'] = pd.to_datetime(df_5th_may_2017['tpep_pickup_datetime2017'])


In [ ]:



# Create an empty graph
G = nx.Graph()
count = 0
# Iterate over DataFrame rows
for index, row in df_total.iterrows():
    # Get the nodes and weight
    pickup_location = row['PULocationID2014']
    dropoff_location = row['DOLocationID2014']
    tip_amount = row['tip_amount2014']
    count += 1
    print(count)
    # If the edge already exists, add the tip amount to the weight
    if G.has_edge(pickup_location, dropoff_location):
        G[pickup_location][dropoff_location]['weight'] += tip_amount
    # If the edge does not exist, create it and set the weight to the tip amount
    else:
        G.add_edge(pickup_location, dropoff_location, weight=tip_amount)


# def make_pairs(temp_list):
#     temp_list=list(temp_list)
#     temp_list.sort()
#     pairs=list(combinations(temp_list,2))
#     return pairs

# def count_pairs(author_list):
#     unique_pairs = {}
#     for author_pair in author_list:
#         sorted_author_pair = sorted(author_pair)
#         sorted_author_pair_tuple = tuple(sorted_author_pair)
#         if sorted_author_pair_tuple in unique_pairs:
#             unique_pairs[sorted_author_pair_tuple] += 1
#         else:
#             unique_pairs[sorted_author_pair_tuple] = 1
#     unique_author_list = [[list(author_pair), count] for author_pair, count in unique_pairs.items()]
#     return unique_author_list

# G = nx.Graph()
# # count = 0
# # for pair in pairs:
# #     G.add_edge(pair[0][0],pair[0][1],attr=pair[1])
# #     count += 1
# #     print(str(count) + " : "+str(len(pairs)))